<a href="https://colab.research.google.com/github/iothouse/SmartProtect/blob/main/Cainco_teste_inicial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Smart Protect - Cainco

In [1]:
# Install Librarie
!pip install cvzone


  Created wheel for cvzone: filename=cvzone-1.5.6-py3-none-any.whl size=18768 sha256=5fda5f8c670c98f4d86277e78b90d2f95cf69f8fb5c87c2b097a67e8bd559bdc
  Stored in directory: /root/.cache/pip/wheels/c1/e8/e9/80f482161ba9f5dcf4832b76ac70540edd11a3136a58445c52
Successfully built cvzone


Libraires

In [ ]:
import cv2
import numpy as np
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
thres = 0.45 # Threshold to detect object
 
cap = cv2.VideoCapture(1)
cap.set(3,1280)
cap.set(4,720)
cap.set(10,70)
 
classNames= []
classFile = ‘coco.names’
with open(classFile,’rt’) as f:
classNames = f.read().rstrip(‘n’).split(‘n’)
 
configPath = ‘ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt’
weightsPath = ‘frozen_inference_graph.pb’
 
net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)
 
while True:
    success,img = cap.read()
    classIds, confs, bbox = net.detect(img,confThreshold=thres)
    print(classIds,bbox)
 
    if len(classIds) != 0:
        for classId, confidence,box in zip(classIds.flatten(),confs.flatten(),bbox):
            cv2.rectangle(img,box,color=(0,255,0),thickness=2)
            cv2.putText(img,classNames[classId-1].upper(),(box[0]+10,box[1]+30),
            cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            cv2.putText(img,str(round(confidence*100,2)),(box[0]+200,box[1]+30),
            cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
 
    cv2.imshow(“Output”,img)
    cv2.waitKey(1)

In [ ]:


thres = 0.45 # Threshold to detect object
nms_threshold = 0.2
cap = cv2.VideoCapture(0)
# cap.set(3,1280)
# cap.set(4,720)
# cap.set(10,150)
 
classNames= []
classFile = ‘coco.names’
with open(classFile,’rt’) as f:
classNames = f.read().rstrip(‘n’).split(‘n’)
 
#print(classNames)
configPath = ‘ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt’
weightsPath = ‘frozen_inference_graph.pb’
 
net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)
 
while True:
    success,img = cap.read()
    classIds, confs, bbox = net.detect(img,confThreshold=thres)
    bbox = list(bbox)
    confs = list(np.array(confs).reshape(1,-1)[0])
    confs = list(map(float,confs))
    #print(type(confs[0]))
    #print(confs)
 
    indices = cv2.dnn.NMSBoxes(bbox,confs,thres,nms_threshold)
    #print(indices)
 
    for i in indices:
        i = i[0]
        box = bbox[i]
        x,y,w,h = box[0],box[1],box[2],box[3]
        cv2.rectangle(img, (x,y),(x+w,h+y), color=(0, 255, 0), thickness=2)
        cv2.putText(img,classNames[classIds[i][0]-1].upper(),(box[0]+10,box[1]+30),
        cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
 
    cv2.imshow(“Output”,img)
    cv2.waitKey(1)
